In [1]:
from cim.loaders import Parameter, ConnectionParameters
from cim.loaders.blazegraph.blazegraph import BlazegraphConnection
from cim.loaders.gridappsd import GridappsdConnection, get_topology_response
from cim.models.distributed_model import DistributedModel
import cim.data_profile as cim
import json


In [2]:
# Blazegraph connection for running inside the container
params = ConnectionParameters([Parameter(key="url", value="http://blazegraph:8080/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params)

In [3]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params)

In [4]:
#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
# feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [5]:
feeder = cim.Feeder(mRID=feeder_mrid)

In [6]:
topology_response = get_topology_response(feeder_mrid)
# with open('topo_message.json') as topo_file:
#     topology_response = json.load(topo_file)


In [7]:
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response)

LinearShuntCompensator
PowerTransformer
LoadBreakSwitch
ACLineSegment
TransformerTank
ACLineSegment
TransformerTank
ACLineSegment
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
LinearShuntCompensator
ACLineSegment
ConnectivityNode
ConnectivityNode
LoadBreakSwitch


In [8]:
# network.switch_areas[0].get_all_attributes(cim.LinearShuntCompensator)
network.switch_areas[1].secondary_areas[0].get_all_attributes(cim.EnergyConsumer)
# network.switch_areas[0].get_all_attributes(cim.WireSpacingInfo)
print('p = ', network.switch_areas[1].secondary_areas[0].typed_catalog[cim.EnergyConsumer]['_3B2021A7-4BFC-418D-9C20-BD6838E52CF8'].p)
print('q = ', network.switch_areas[1].secondary_areas[0].typed_catalog[cim.EnergyConsumer]['_3B2021A7-4BFC-418D-9C20-BD6838E52CF8'].q)
print(network.switch_areas[1].secondary_areas[0].typed_catalog[cim.EnergyConsumer]['_3B2021A7-4BFC-418D-9C20-BD6838E52CF8'].Terminals)

IndexError: list index out of range

In [10]:
network.get_all_attributes(cim.ACLineSegment)
network.get_all_attributes(cim.ACLineSegmentPhase)
network.get_all_attributes(cim.WireSpacingInfo)
network.get_all_attributes(cim.WirePosition)

In [12]:
network.get_all_attributes(cim.TapeShieldCableInfo)

In [13]:
json.loads(network.__dumps__(cim.TapeShieldCableInfo))

{'_9EE93180-5EA6-42FC-8EC5-35B82D4F4D8C': {'mRID': '_9EE93180-5EA6-42FC-8EC5-35B82D4F4D8C',
  'aliasName': '',
  'description': '',
  'name': 'ts_1/0',
  'Names': [],
  'Assets': [],
  'PowerSystemResources': [],
  'coreRadius': '0',
  'coreStrandCount': '0',
  'gmr': '0.00338328',
  'insulated': 'true',
  'insulationMaterial': 'crosslinkedPolyethylene',
  'insulationThickness': '0.005588',
  'material': '',
  'rAC25': '0.00060274654',
  'rAC50': '0.00060274654',
  'rAC75': '0.00060274654',
  'radius': '0.0046736',
  'ratedCurrent': '165',
  'rDC20': '0.00059092798',
  'sizeDescription': 'TSData_ts_1/0',
  'strandCount': '0',
  'ACLineSegmentPhases': ['_B65BD02D-760B-451F-81C6-C8FA78478899'],
  'WirePhaseInfo': [],
  'constructionKind': '',
  'diameterOverCore': '',
  'diameterOverInsulation': '0.020828',
  'diameterOverJacket': '0.026924',
  'diameterOverScreen': '0.022098',
  'isStrandFill': 'false',
  'nominalTemperature': '90',
  'outerJacketKind': '',
  'sheathAsNeutral': 'true',


In [ ]:
import time
tic = time.time()
obj = bg.create_default_instances(feeder.mRID, ["_E72A5E73-15F0-4FE5-BFFE-D4C9491106DA",  "_64D89EF8-60E7-486A-8BA8-27A2DDCB28C7" ,  "_7C4CB7EC-4ED3-497F-9106-B1B8364E4627" , "_CBFEC50B-A94A-4B42-A83A-45FC25B11F9D" , "_F0677038-7445-4F72-BB1C-FD9B406B2E1F" , "_4646B654-2416-424E-B953-6D81AFA95358"] )
print(obj)
print(time.time()-tic)

In [27]:
network.typed_catalog[cim.WireSpacingInfo]['_E804FC3A-4DE8-4497-B83D-EBE8974C791B'].ACLineSegments[0].WireSpacingInfo.mRID

'_E804FC3A-4DE8-4497-B83D-EBE8974C791B'

In [ ]:
network

In [20]:
cim.ACLineSegment().__reduce__()

(<function copyreg._reconstructor(cls, base, state)>,
 (cim.data_profile.gridappsd_cim_profile.ACLineSegment, object, None),
 {'mRID': None,
  'aliasName': None,
  'description': None,
  'name': None,
  'Names': [],
  'AssetDatasheet': None,
  'Assets': [],
  'Controls': [],
  'Location': None,
  'Measurements': [],
  'inService': None,
  'Faults': [],
  'OperationalLimitSet': [],
  'BaseVoltage': None,
  'SvStatus': [],
  'Terminals': [],
  'length': None,
  'b0ch': None,
  'bch': None,
  'g0ch': None,
  'gch': None,
  'r': None,
  'r0': None,
  'x': None,
  'x0': None,
  'ACLineSegmentPhases': [],
  'PerLengthImpedance': None,
  'WireSpacingInfo': None})

In [ ]:
def pretty_print(clas, indent=0):
    print(' ' * indent +  type(clas).__name__ +  ':')
    indent += 4
    for k,v in clas.__dict__.items():
        if '__dict__' in dir(v):
            pretty_print(v,indent)
        else:
            print(' ' * indent +  k + ': ' + str(v))

In [ ]:
pretty_print(network.typed_catalog[cim.WireSpacingInfo]['_E804FC3A-4DE8-4497-B83D-EBE8974C791B'])

In [ ]:
pretty_print(network.typed_catalog[cim.ACLineSegmentPhase]['_CE2AFB05-A1B1-4082-B4EB-2AD8A1D3422D'])

In [ ]:
value = 'abc'
separator = ';'
values = value.split(separator)
obj_list = []
obj_list.extend(values)
obj_list

In [ ]:
len(values)

In [ ]:
pretty_print(cim.WireInsulationKind)

In [ ]:
network.get_all_attributes(cim.TapeShieldCableInfo)

In [ ]:
network.typed_catalog[cim.TapeShieldCableInfo]

In [ ]:
class_name = 'ACLineSegment'
cls = class_name
class_type = eval(f"cim.{cls}")
network.typed_catalog[class_type]

In [ ]:
network.__dumps__(cim.ACLineSegment)

In [ ]:
value = cim.ACLineSegment()
t = type(value)


In [ ]:
type(t)

In [ ]:
if type(type(value)) is type:
    print('yes')